In [292]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

import ipykernel.iostream
import sys

import urllib3.contrib.pyopenssl

In [273]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Attribution: https://stackoverflow.com/questions/43846689/sslerror-bad-handshake-syscallerror54-econnreset

In [24]:
links = pd.read_csv('../data/links.csv', names = ['link'], skiprows=1)

In [25]:
links

,link
0,https://onebite.app/restaurant/pizza-hut-adana...
1,https://onebite.app/restaurant/del-sole-01-472...
2,https://onebite.app/restaurant/lava-lounge-piz...
3,https://onebite.app/restaurant/pizzaria-cosa-n...
4,https://onebite.app/restaurant/olivia-tjuvholm...
...,...
50582,https://onebite.app/restaurant/very-italian-pi...
50583,https://onebite.app/restaurant/donatellos-leid...
50584,https://onebite.app/restaurant/santa-lucia-lim...
50585,https://onebite.app/restaurant/pizze-pronto-sc...


In [302]:
start = time.time()

dave_scores = []
user_scores = []
name = []
address = []

links2 = links

for link in list(links2['link']):

    clear_output()
    print(link)
    r = requests.get(link)
    bs = BeautifulSoup(r.text)
    
    if r.status_code == 200:

        temp = []

        for elem in bs.find_all('a'):
            if 'jsx-84601126' in (elem.attrs['class']):
                temp.append(elem.text)

        name.append(temp[0])
        address.append(temp[1])

        temp2 = []

        for elem in bs.find_all('span'):
            if 'score' in (elem.attrs['class']):
                temp2.append(elem.text)

        dave_scores.append(temp2[0])
        user_scores.append(temp2[1])
        
    else:
        pass

    time.sleep(0.25)
    
end = time.time()
    
print(end-start)

https://onebite.app/restaurant/luigia-zurich-zh-851254c3
102.5960841178894


In [303]:
df = pd.DataFrame({'restaurant_name': name,
                   'dave_scores': dave_scores,
                   'user_scores': user_scores,
                   'address': address})

df

,restaurant_name,dave_scores,user_scores,address
0,Pizza Hut,0.0,2.7,"213. Sok., Adana, 01"
1,Del sole,0.0,7.3,"字ニセコ483-1, 虻田郡, 01"
2,Lava Lounge Pizza Niseko,0.0,4.2,"倶知安町字山田191-19, 虻田郡, 01"
3,Pizzaria Cosa Nostra,0.0,8.5,"R. Custódio Brás Pacheco, 1, Vila Nova de Milf..."
4,Olivia Tjuvholmen,0.0,8.4,"Bryggegangen 4, Oslo, 03"
...,...,...,...,...
50582,Very Italian Pizza,0.0,7.4,"Turfmarkt 8, Leiden, ZH"
50583,Donatello's,0.0,6.2,"Haarlemerstraat 20, Leiden, ZH"
50584,Santa Lucia Limmatplatz,0.0,6.8,"Luisenstrasse 31, Zürich, ZH"
50585,Pizze Pronto,0.0,7.7,"Schoolstraat 1, Schiedam, ZH"


Let's check the values.
First we convert the numeric columns to numeric.

In [319]:
df['dave_scores'] = pd.to_numeric(df['dave_scores'], downcast="float")
df['user_scores'] = pd.to_numeric(df['user_scores'], downcast="float")

Check the average of each column. 
The column with Dave's scores should rule out the reviews that are 0, as these are restaurants that he hasn't reviewed. 

In [321]:
df[df['dave_scores'] != 0]['dave_scores'].mean()

6.656379699707031

In [322]:
df['user_scores'].mean()

6.473654747009277

User scores are a little lower than Dave's scores which is surprising, but they both seem like reasonable averages. 

In [323]:
df.to_csv("../data/reviews.csv")

,link
0,https://onebite.app/restaurant/pizza-hut-adana...
1,https://onebite.app/restaurant/del-sole-01-472...
2,https://onebite.app/restaurant/lava-lounge-piz...
3,https://onebite.app/restaurant/pizzaria-cosa-n...
4,https://onebite.app/restaurant/olivia-tjuvholm...
...,...
50582,https://onebite.app/restaurant/very-italian-pi...
50583,https://onebite.app/restaurant/donatellos-leid...
50584,https://onebite.app/restaurant/santa-lucia-lim...
50585,https://onebite.app/restaurant/pizze-pronto-sc...
